###############################################################################################################

###################################################################################################

In [1]:
import time
import hashlib
import json
import pandas as pd
from collections import defaultdict
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- Blockchain Classes ---
class Block:
    def __init__(self, index, transactions, timestamp, previous_hash):
        self.index = index
        self.transactions = transactions
        self.timestamp = timestamp
        self.previous_hash = previous_hash
        self.nonce = 0
        self.hash = self.compute_hash()

    def compute_hash(self):
        block_string = json.dumps(self.__dict__, sort_keys=True, default=str)
        return hashlib.sha256(block_string.encode()).hexdigest()

class Blockchain:
    def __init__(self):
        self.chain = []
        self.pending_transactions = []
        self.voters = set()
        self.create_genesis_block()

    def create_genesis_block(self):
        genesis_block = Block(0, [], time.time(), "0")
        self.chain.append(genesis_block)

    def add_transaction(self, voter_id, candidate, receipt_id):
        if voter_id in self.voters:
            return False # Already voted
        tx = {
            'voter_id': voter_id,
            'candidate': candidate,
            'receipt_id': receipt_id,  # <-- Add receipt ID for verifiability
            'timestamp': time.time()
        }
        self.pending_transactions.append(tx)
        self.voters.add(voter_id)
        return True

    def mine(self):
        if not self.pending_transactions:
            return False
        last_block = self.chain[-1]
        new_block = Block(
            index=len(self.chain),
            transactions=self.pending_transactions,
            timestamp=time.time(),
            previous_hash=last_block.hash
        )
        self.chain.append(new_block)
        self.pending_transactions = []
        return True

    def get_all_transactions(self):
        txs = []
        for block in self.chain[1:]: # skip genesis
            txs.extend(block.transactions)
        return txs

    def get_results(self):
        results = defaultdict(int)
        for tx in self.get_all_transactions():
            results[tx['candidate']] += 1
        return results

    def get_block_info(self):
        info = []
        for block in self.chain:
            info.append({
                'index': block.index,
                'timestamp': time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(block.timestamp)),
                'transactions': block.transactions,
                'hash': block.hash,
                'previous_hash': block.previous_hash,
            })
        return info

blockchain = Blockchain()


In [2]:
# Global State and Synchronization Helpers

# --- Global State ---
candidate_list = []
vote_counts = {}
voter_details = []

# --- Synchronization Helper ---
def sync_qr_widgets():
    scan_dropdown.options = candidate_list.copy()
    for cand in candidate_list:
        if cand not in vote_counts:
            vote_counts[cand] = 0
    for cand in list(vote_counts.keys()):
        if cand not in candidate_list:
            del vote_counts[cand]
    show_qr_results()
    show_qr_codes()
    update_vote_options()


In [3]:
# Dashboard Tab
from IPython.display import display, clear_output, HTML

dashboard_output_admin = widgets.Output()
dashboard_output_voter = widgets.Output()

def show_dashboard(output_widget):
    with output_widget:
        clear_output()
        num_candidates = len(candidate_list)
        num_voters = len(voter_details)
        total_votes = sum(blockchain.get_results().values())
        turnout = f"{(total_votes/num_voters*100):.1f}%" if num_voters else "0%"
        last_block = blockchain.chain[-1]
        last_block_info = f"Block #{last_block.index} | Hash: {last_block.hash[:10]}..."
        dashboard_html = f"""
        <div style='border:2px solid #0074D9; border-radius:10px; padding:20px; margin-bottom:20px; background:#f9f9f9;'>
          <h2 style='color:#0074D9;'>🗳️ Blockchain Voting System Dashboard</h2>
          <b>Election Status:</b> <span style='color:green;'>Open</span><br>
          <b>Candidates:</b> {num_candidates} &nbsp; | &nbsp;
          <b>Registered Voters:</b> {num_voters} &nbsp; | &nbsp;
          <b>Votes Cast:</b> {total_votes} &nbsp; | &nbsp;
          <b>Turnout:</b> {turnout}<br>
          <b>Last Block:</b> {last_block_info}<br>
          <hr>
          <b>Instructions:</b><br>
          <ul>
            <li><b>Admins:</b> Use the <span style='color:#0074D9;'>Admin Section</span> to manage candidates, monitor results, and audit the blockchain.</li>
            <li><b>Voters:</b> Use the <span style='color:#2ECC40;'>Voter Section</span> to register, vote, and verify your receipt.</li>
          </ul>
        </div>
        """
        display(HTML(dashboard_html))

show_dashboard(dashboard_output_admin)
show_dashboard(dashboard_output_voter)

dashboard_tab_admin = widgets.VBox([dashboard_output_admin])
dashboard_tab_voter = widgets.VBox([dashboard_output_voter])


In [4]:
# Admin Tab

candidate_input = widgets.Text(
    value='',
    placeholder='Enter candidate/option name',
    description='Option:',
)
candidate_list_box = widgets.Select(
    options=candidate_list,
    description='Options:',
    rows=5,
    disabled=False
)
add_candidate_button = widgets.Button(description="Add Option", button_style='success')
remove_candidate_button = widgets.Button(description="Remove Selected", button_style='danger')
clear_all_button = widgets.Button(description="Clear All (Reset Election)", button_style='danger')
admin_output = widgets.Output()

def on_add_candidate(b):
    with admin_output:
        clear_output()
        option = candidate_input.value.strip()
        if option and option not in candidate_list:
            candidate_list.append(option)
            candidate_list_box.options = candidate_list
            print(f"✅ Added: {option}")
            sync_qr_widgets()
        else:
            print("❌ Enter a unique, non-empty option.")

def on_remove_candidate(b):
    with admin_output:
        clear_output()
        selected = candidate_list_box.value
        if selected in candidate_list:
            candidate_list.remove(selected)
            candidate_list_box.options = candidate_list
            print(f"❌ Removed: {selected}")
            sync_qr_widgets()
        else:
            print("No option selected.")

def clear_all(b=None):
    candidate_list.clear()
    candidate_list_box.options = candidate_list
    vote_counts.clear()
    voter_details.clear()
    global blockchain
    blockchain = Blockchain()
    sync_qr_widgets()
    with admin_output:
        clear_output()
        print("✅ All data cleared. Ready for new election.")

add_candidate_button.on_click(on_add_candidate)
remove_candidate_button.on_click(on_remove_candidate)
clear_all_button.on_click(clear_all)

admin_tab = widgets.VBox([
    widgets.HTML("<h3>Admin: Manage Voting Options</h3>"),
    candidate_input, add_candidate_button, remove_candidate_button, clear_all_button,
    candidate_list_box, admin_output
])


In [5]:
# Voter ID

import re

voter_id_input = widgets.Text(placeholder='Unique ID', description='Voter ID:')
voter_name_input = widgets.Text(placeholder='Full Name', description='Name:')
voter_domain_input = widgets.Text(placeholder='Domain (e.g., IT, HR)', description='Domain:')
voter_email_input = widgets.Text(placeholder='Email', description='Email:')
voter_lang_input = widgets.Dropdown(
    options=['English', 'Hindi', 'Spanish'],
    description='Language:'
)
vote_candidate_radio = widgets.RadioButtons(options=candidate_list, description='Vote:')
vote_button = widgets.Button(description="Cast Vote", button_style='primary')
clear_voters_button = widgets.Button(description="Clear Voters", button_style='danger')
voter_output = widgets.Output()

def update_vote_options(*args):
    vote_candidate_radio.options = candidate_list

def is_valid_email(email):
    # Simple regex for email validation
    pattern = r'^[\w\.-]+@[\w\.-]+\.\w+$'
    return re.match(pattern, email) is not None

def on_vote(b):
    with voter_output:
        clear_output()
        voter_id = voter_id_input.value.strip()
        name = voter_name_input.value.strip()
        domain = voter_domain_input.value.strip()
        email = voter_email_input.value.strip()
        lang = voter_lang_input.value
        candidate = vote_candidate_radio.value
        
        if not (voter_id and name and domain and email and candidate):
            print("❌ Please fill all fields and select a candidate.")
            return
        
        if not is_valid_email(email):
            print("❌ Please enter a valid email address (e.g., abcd@gmail.com).")
            return
        
        if any(v['Voter ID'] == voter_id for v in voter_details):
            print("❌ This Voter ID has already voted.")
            return
        
        # Generate unique receipt ID (hash of voter ID + timestamp)
        receipt_id = hashlib.sha256(f"{voter_id}{time.time()}".encode()).hexdigest()
        
        # Add to blockchain (modify transaction format)
        success = blockchain.add_transaction(
            voter_id=voter_id,
            candidate=candidate,
            receipt_id=receipt_id  # <-- New field
        )
        
        if success:
            blockchain.mine()
            voter_details.append({
                'Voter ID': voter_id,
                'Name': name,
                'Domain': domain,
                'Email': email,
                'Language': lang,
                'Voted For': candidate,
                'Receipt ID': receipt_id,  # <-- Track receipt ID
                'Timestamp': time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(time.time()))
            })
            print(f"✅ Vote cast for {candidate}.")
            print(f"Your receipt ID: {receipt_id}")
            show_results()
        else:
            print("❌ This Voter ID has already voted.")

def clear_voters(b=None):
    voter_details.clear()
    with voter_output:
        clear_output()
        print("✅ All voter data cleared.")
    show_voter_table()

vote_button.on_click(on_vote)
clear_voters_button.on_click(clear_voters)

voting_tab = widgets.VBox([
    widgets.HTML("<h3>Voter Registration & Voting</h3>"),
    voter_id_input, voter_name_input, voter_domain_input, voter_email_input, voter_lang_input,
    vote_candidate_radio, vote_button, clear_voters_button, voter_output
])


In [6]:
voter_table_output = widgets.Output()

def show_voter_table():
    with voter_table_output:
        clear_output()
        if not voter_details:
            print("No voters yet.")
        else:
            df = pd.DataFrame(voter_details)
            display(df)

voter_table_refresh = widgets.Button(description="Refresh Table", button_style='info')
voter_table_refresh.on_click(lambda b: show_voter_table())
voter_table_tab = widgets.VBox([
    widgets.HTML("<h3>All Voter Details</h3>"),
    voter_table_refresh, voter_table_output
])
show_voter_table()


In [7]:
# Results Tab

results_output = widgets.Output()

def show_results():
    with results_output:
        clear_output()
        results = blockchain.get_results()
        if not results:
            print("No votes yet.")
        else:
            print("Voting Results:")
            for cand in candidate_list:
                print(f"{cand}: {results.get(cand,0)} votes")

results_refresh = widgets.Button(description="Refresh Results", button_style='info')
results_refresh.on_click(lambda b: show_results())
results_tab = widgets.VBox([
    widgets.HTML("<h3>Voting Results</h3>"),
    results_refresh, results_output
])
show_results()


In [8]:
# Blockchain Explorer Tab

blockchain_output = widgets.Output()

def show_blockchain():
    with blockchain_output:
        clear_output()
        info = blockchain.get_block_info()
        for block in info:
            print(f"Block {block['index']} | Hash: {block['hash'][:10]}... | Prev: {block['previous_hash'][:10]}...")
            print(f"  Timestamp: {block['timestamp']}")
            print(f"  Transactions: {json.dumps(block['transactions'], indent=2)}\n")

blockchain_refresh = widgets.Button(description="Refresh Blockchain", button_style='info')
blockchain_refresh.on_click(lambda b: show_blockchain())
blockchain_tab = widgets.VBox([
    widgets.HTML("<h3>Blockchain Details</h3>"),
    blockchain_refresh, blockchain_output
])
show_blockchain()


In [9]:
# QR Code Tab

import qrcode
from PIL import Image

qr_output = widgets.Output()

def show_qr_codes():
    with qr_output:
        clear_output()
        if not candidate_list:
            print("No candidates yet.")
        else:
            for candidate in candidate_list:
                qr = qrcode.QRCode(version=1, box_size=4, border=2)
                qr.add_data(candidate)
                qr.make(fit=True)
                img = qr.make_image(fill='black', back_color='white')
                print(f"QR Code for {candidate}:")
                display(img)

refresh_qr_btn = widgets.Button(description="Refresh QR Codes", button_style='info')
refresh_qr_btn.on_click(lambda b: show_qr_codes())

qr_tab = widgets.VBox([
    widgets.HTML("<h3>Candidate QR Codes</h3>"),
    refresh_qr_btn,
    qr_output
])
show_qr_codes()


In [10]:
# QR Scan Tab

scan_dropdown = widgets.Dropdown(
    options=candidate_list.copy(),
    description="Scanned QR:",
)
scan_button = widgets.Button(description="Simulate QR Scan", button_style='success')
scan_output = widgets.Output()

def on_scan_vote(b):
    with scan_output:
        clear_output()
        candidate = scan_dropdown.value
        if candidate:
            vote_counts[candidate] += 1
            print(f"✅ Vote registered for {candidate} via QR scan!")
            show_qr_results()
        else:
            print("❌ No candidate selected.")

scan_button.on_click(on_scan_vote)

scan_tab = widgets.VBox([
    widgets.HTML("<h3>Simulate QR Code Scanning & Vote Counting</h3>"),
    scan_dropdown, scan_button, scan_output
])


In [11]:
# QR Vote Count Tab

qr_results_output = widgets.Output()

def show_qr_results():
    with qr_results_output:
        clear_output()
        print("Live Vote Counts (QR Scanning):")
        for cand in candidate_list:
            print(f"{cand}: {vote_counts.get(cand, 0)}")

qr_results_refresh = widgets.Button(description="Refresh QR Vote Counts", button_style='info')
qr_results_refresh.on_click(lambda b: show_qr_results())

qr_results_tab = widgets.VBox([
    widgets.HTML("<h3>Live QR Scan Vote Counts</h3>"),
    qr_results_refresh, qr_results_output
])
show_qr_results()


In [12]:
# --- Verification Tab ---
receipt_input = widgets.Text(placeholder='Enter Receipt ID', description='Receipt ID:')
verify_button = widgets.Button(description="Verify Vote", button_style='info')
verification_output = widgets.Output()

def on_verify(b):
    with verification_output:
        clear_output()
        receipt_id = receipt_input.value.strip()
        if not receipt_id:
            print("❌ Enter a receipt ID.")
            return
        print('/n')
        # Search blockchain for receipt ID
        vote_found = False
        for block in blockchain.chain:
            for tx in block.transactions:
                if tx.get('receipt_id') == receipt_id:
                    print("✅ Your vote has been counted and verified!")
                    vote_found = True
                    break
            if vote_found:
                break
        if not vote_found:
            print("❌ Receipt ID not found. Vote not counted or invalid ID.")

verify_button.on_click(on_verify)

verification_tab = widgets.VBox([
    widgets.HTML("<h3>Verify Your Vote</h3>"),
    receipt_input,
    verify_button,
    verification_output
])


In [15]:
# --- Admin Subtabs ---
admin_tabs = widgets.Tab(children=[
    dashboard_tab_admin,  # Admin dashboard tab
    admin_tab, voter_table_tab, results_tab, blockchain_tab, ai_tab, qr_tab, scan_tab, qr_results_tab, verification_tab
])
admin_tabs.set_title(0, 'Dashboard')
admin_tabs.set_title(1, 'Admin')
admin_tabs.set_title(2, 'Voter Table')
admin_tabs.set_title(3, 'Results')
admin_tabs.set_title(4, 'Blockchain')
admin_tabs.set_title(5, 'AI Fraud Detection')
admin_tabs.set_title(6, 'QR Codes')
admin_tabs.set_title(7, 'QR Scan')
admin_tabs.set_title(8, 'QR Vote Counts')
section_tabs = widgets.Tab(children=[admin_tabs, voter_tabs])
section_tabs.set_title(0, 'Admin Section')
section_tabs.set_title(1, 'Voter Section')

admin_tabs.set_title(9, 'Verify Vote')

# --- Voter Subtabs ---
voter_tabs = widgets.Tab(children=[
    dashboard_tab_voter,  # Voter dashboard tab
    voting_tab, results_tab, qr_tab, scan_tab, verification_tab   
])
voter_tabs.set_title(0, 'Dashboard')
voter_tabs.set_title(1, 'Vote')
voter_tabs.set_title(2, 'Results')
voter_tabs.set_title(3, 'QR Codes')
voter_tabs.set_title(4, 'QR Scan')
voter_tabs.set_title(5, 'Verify Vote')

section_tabs = widgets.Tab(children=[admin_tabs, voter_tabs])
section_tabs.set_title(0, 'Admin Section')
section_tabs.set_title(1, 'Voter Section')

display(section_tabs)


In [ ]:
sync_qr_widgets()